<a href="https://colab.research.google.com/github/YuvanBharathi13/DeepLearning/blob/main/4_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Knowing the data

## unzipping the file

In [2]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2025-06-06 15:52:02--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.207, 74.125.201.207, 74.125.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   210MB/s    in 0.8s    

2025-06-06 15:52:03 (210 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



## walking through the directory

In [3]:
import os
for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} folders and {len(filenames)} images in directory {dirpath}")

There are 2 folders and 0 images in directory 10_food_classes_10_percent
There are 10 folders and 0 images in directory 10_food_classes_10_percent/train
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/sushi
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/steak
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/ramen
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/fried_rice
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/chicken_curry
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/hamburger
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/grilled_salmon
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/chicken_wings
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/ice_cream
There are 0 folders and 75 images i

## class names

In [4]:
import pathlib
data_dir = pathlib.Path('10_food_classes_10_percent/train')
class_names = np.array(sorted(item.name for item in data_dir.glob('*')))
class_names

array(['chicken_curry', 'chicken_wings', 'fried_rice', 'grilled_salmon',
       'hamburger', 'ice_cream', 'pizza', 'ramen', 'steak', 'sushi'],
      dtype='<U14')

# Pre-processing data

In [8]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
